# Import Object

In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

from models import Seq2Seq, Encoder, Decoder
from train_model import train_seq2seq_net

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare Data

In [4]:
train_x = pd.read_csv("./data/2_X_train.csv").values
train_y = pd.read_csv("./data/2_Y_train.csv").values.reshape(-1)

sequence_length = 16
num_days = int(train_x.shape[0] / sequence_length)
feature_size = train_x.shape[1]
continuous_feature_size = 8
test_size = 0.1

train_x = train_x.reshape(num_days, sequence_length, feature_size)
train_y = train_y.reshape(num_days, sequence_length, 1)
train_x = train_x[:, ::-1, :]
train_x, test_x, train_y, test_y = train_test_split(train_x,
                                                    train_y,
                                                    test_size=test_size,
                                                    shuffle=True)

train_num_days = int(num_days * (1-test_size))
test_num_days = int(num_days - train_num_days)

train_x = train_x.reshape(train_num_days * sequence_length, feature_size)
test_x = test_x.reshape(test_num_days * sequence_length, feature_size)

ss = preprocessing.StandardScaler()
ss.fit(train_x[:, :continuous_feature_size])
train_x[:, :continuous_feature_size] = \
    ss.transform(train_x[:, :continuous_feature_size])

test_x[:, :continuous_feature_size] = \
    ss.transform(test_x[:, :continuous_feature_size])

train_x = train_x.reshape(train_num_days, sequence_length, feature_size)
test_x = test_x.reshape(test_num_days, sequence_length, feature_size)

train_x = torch.tensor(train_x, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

train_x = train_x.to(device)
train_y = train_y.to(device)
test_x = test_x.to(device)
test_y = test_y.to(device)

train_ds = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_ds, batch_size=6, shuffle=True)

# Set Free Parameters

In [5]:
input_size_encoder = 13
input_size_decoder = 1
num_layers = 1
hidden_size = 50
num_classes = 2
learning_rate = 0.001
num_epochs = 50
dropout_ratio = 0.5

# Instantiate Model, Criterion, Optimizer

In [6]:
encoder_net = Encoder(input_size_encoder, hidden_size,
                      num_layers, dropout_ratio).to(device)
decoder_net = Decoder(input_size_decoder, hidden_size,
                      num_classes, num_layers, dropout_ratio).to(device)

model = Seq2Seq(encoder_net, decoder_net).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/home/higuchi-lab/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


# Training

In [7]:
model = train_seq2seq_net(model, criterion, optimizer,
                          num_epochs, train_loader, device)

# Accuracy

In [8]:
model.eval()
decoder_y = test_y[:, :-1, :]

decoder_t = test_y[:, 1:, :]
decoder_t = decoder_t.reshape(-1)

scores = model.generate(test_x, decoder_y, decoder_y.shape[1], device)
pred_y = scores.argmax(axis=2)
pred_y = pred_y.reshape(-1)

print(sum(pred_y == decoder_t) / int(decoder_t.shape[0]))

tensor(0.8933, device='cuda:0')
